### Задача 1 (10 баллов)

Вы разрабатываете игру, в которой главный герой — археолог и исследователь древних цивилизаций. Его цель — собрать и исследовать артефакты, которые разбросаны по разным древним руинам. Ваша задача — создать класс ArtifactCollector, который будет управлять коллекцией артефактов. Эта коллекция должна быть итерируемой, поддерживать сортировку и фильтрацию артефактов по разным критериям.

Требования:

Создайте класс Artifact, который представляет артефакт. Артефакт должен иметь следующие свойства:

- name — название артефакта.
- age — возраст артефакта (в годах).
- origin — происхождение артефакта (например, "Egypt", "Greece", "Aztec").
- rarity — редкость артефакта (например, "common", "rare", "legendary").

Создайте класс ArtifactCollector, который управляет коллекцией артефактов. Коллекция должна:

- Поддерживать добавление артефактов.
- Быть итерируемой (по умолчанию артефакты выводятся в порядке добавления).
- Поддерживать сортировку по возрасту артефактов с помощью метода sort_by_age().
- Поддерживать фильтрацию артефактов по происхождению с помощью метода filter_by_origin(origin).
- Поддерживать фильтрацию по редкости с помощью метода filter_by_rarity(rarity).
- Метод reversed должен возвращать артефакты в обратном порядке (например, чтобы археолог мог просмотреть последние найденные).

Дополнительное задание (+балл):

- Реализуйте возможность находить самый древний артефакт с помощью метода find_oldest().
- Добавьте возможность находить артефакт с определённым именем через метод find_by_name(name).

Пример использования:


#### Создаем артефакты

    mask = Artifact("Golden Mask", 3000, "Egypt", "legendary")
    vase = Artifact("Ancient Vase", 2000, "Greece", "rare")
    statue = Artifact("Stone Statue", 1500, "Aztec", "common")
    amulet = Artifact("Amulet of Anubis", 5000, "Egypt", "legendary")

#### Создаем коллекционера артефактов и добавляем в коллекцию

    collector = ArtifactCollector()
    collector.add_artifact(mask)
    collector.add_artifact(vase)
    collector.add_artifact(statue)
    collector.add_artifact(amulet)

#### Итерация по коллекции

    print("Все артефакты:")
    for artifact in collector:
        print(artifact.name)

#### Сортировка по возрасту

    print("\nАртефакты по возрасту:")
    collector.sort_by_age()
    for artifact in collector:
        print(f"{artifact.name}: {artifact.age} лет")

#### Фильтрация по происхождению

    print("\nАртефакты из Египта:")
    for artifact in collector.filter_by_origin("Egypt"):
        print(artifact.name)

#### Самый древний артефакт

    oldest = collector.find_oldest()
    print(f"\nСамый древний артефакт: {oldest.name} ({oldest.age} лет)")

#### Поиск по имени

    artifact = collector.find_by_name("Ancient Vase")
    if artifact:
        print(f"\nАртефакт найден: {artifact.name}, {artifact.age} лет, {artifact.origin}")
    else:
        print("\nАртефакт не найден")

#### Обратный порядок итерации

    print("\nАртефакты в обратном порядке:")
    for artifact in reversed(collector):
        print(artifact.name)
        
Подсказки:

Магические методы \_\_iter\_\_ и \_\_reversed\_\_ помогут сделать коллекцию итерируемой.

Для сортировки и фильтрации используйте генераторы или списковые включения.

Для поиска используйте стандартные методы работы с коллекциями (max(), min()).

In [14]:
class Artifact:
    def __init__(self, name, age : int, origin, rarity):
        self.name = name
        self.origin = origin
        self.age = age
        self.rarity = rarity

    def __repr__(self):
        return f"Artifact {self.name}, age {self.age}, origin {self.origin}, {self.rarity}"
    
    
    def __str__(self):
        return f"{self.name}"
    

In [52]:
class ArtifactCollector:
    def __init__(self):
        self.artifacts = []


    def add_artifacts(self, *artifacts):
        for a in artifacts:
            self.artifacts.append(a)


    def __iter__(self):
        for i in range(len(self.artifacts)):
            yield self.artifacts[i]

    def __reversed__(self):
            return iter(self.artifacts[::-1])
    

    def find_oldest(self):
        '''Возвращает артефакт с самым большим значением возраста'''
        o = max([a for a in self.artifacts], key=lambda x: x.age)
        return o
    
    
    def find_by_name(self, name):
        '''Возвращает первый из добавленных в коллекцию артефакт с переданным названием'''
        artifact = next((a for a in self.artifacts if a.name == name), None)
        return artifact
    
    def filter_by_origin(self, origin):
         '''Возвращает список артефактов, происхождение которых совпадает с переданным в функцию'''
         filtered = [a for a in self.artifacts if a.origin == origin]
         return filtered

    def filter_by_rarity(self, rarity):
         '''Возвращает список артефактов, редкость которых совпадает с переданной в функцию'''
         filtered = [a for a in self.artifacts if a.rarity == rarity]
         return filtered

    def sort_by_age(self, descedent : bool=False):
        '''Сортирует артефакты в коллекции по возрасту в порядке возрастания (по умолчанию) или убывания (descedent=True)'''
        filtered = sorted([a for a in self.artifacts], key=lambda x: x.age, reverse=descedent)
        return filtered




In [53]:
mask = Artifact("Golden Mask", 3000, "Egypt", "legendary")
vase = Artifact("Ancient Vase", 2000, "Greece", "rare")
statue = Artifact("Stone Statue", 1500, "Aztec", "common")
amulet = Artifact("Amulet of Anubis", 5000, "Egypt", "legendary")
collector = ArtifactCollector()
collector.add_artifacts(mask, vase, statue, amulet)

In [48]:
print("Все артефакты:")
for artifact in collector:
    print(artifact.name)

Все артефакты:
Golden Mask
Ancient Vase
Stone Statue
Amulet of Anubis


In [49]:
print("\nАртефакты по возрасту:")
for artifact in collector.sort_by_age():
    print(f"{artifact.name}: {artifact.age} лет")




Артефакты по возрасту:
Stone Statue: 1500 лет
Ancient Vase: 2000 лет
Golden Mask: 3000 лет
Amulet of Anubis: 5000 лет


In [50]:
print("\nАртефакты по возрасту:")
for artifact in collector.sort_by_age(descedent=True):
    print(f"{artifact.name}: {artifact.age} лет")


Артефакты по возрасту:
Amulet of Anubis: 5000 лет
Golden Mask: 3000 лет
Ancient Vase: 2000 лет
Stone Statue: 1500 лет


In [54]:
print("\nАртефакты из Египта:")
for artifact in collector.filter_by_origin("Egypt"):
    print(artifact.name)


Артефакты из Египта:
Golden Mask
Amulet of Anubis


In [59]:
print("\nРедкие артефакты:")
for artifact in collector.filter_by_rarity("rare"):
    print(artifact.name)


Редкие артефакты:
Ancient Vase


In [55]:
oldest = collector.find_oldest()
print(f"\nСамый древний артефакт: {oldest.name} ({oldest.age} лет)")


Самый древний артефакт: Amulet of Anubis (5000 лет)


In [56]:
artifact = collector.find_by_name("Golden Mask")
if artifact:
    print(f"\nАртефакт найден: {artifact.name}, {artifact.age} лет, {artifact.origin}")
else:
    print("\nАртефакт не найден")


Артефакт найден: Golden Mask, 3000 лет, Egypt


In [57]:
artifact = collector.find_by_name("Zolotaya Maska")
if artifact:
    print(f"\nАртефакт найден: {artifact.name}, {artifact.age} лет, {artifact.origin}")
else:
    print("\nАртефакт не найден")


Артефакт не найден


In [58]:
print("\nАртефакты в обратном порядке:")
for artifact in reversed(collector):
    print(artifact.name)


Артефакты в обратном порядке:
Amulet of Anubis
Stone Statue
Ancient Vase
Golden Mask
